In [ ]:
import pandas as pd
import pyodbc
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# Path to Excel file
excel_path = r"D:\Documents\Enrollment\dashboard_datasets.xlsx"

# Load ALL sheets into a dictionary {sheet_name: DataFrame}
sheets = pd.read_excel(excel_path, sheet_name=None)

# Connect to SQL Server
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=andreakings;'
    'DATABASE=university_db;'
    'Trusted_Connection=yes;'
)
cursor = conn.cursor()

# Map sheet names to table names + insert SQL
sheet_to_table = {
    # "enrolment records": (
    #     "enrollment_records",
    #     """
    #     INSERT INTO enrollment_records 
    #     (student_id, enrollment_id, enrollment_year, enrollment_status, admission_type, program, faculty)
    #     VALUES (?, ?, ?, ?, ?, ?, ?)
    #     """
    # ),
    # "Courses": (
    #     "courses",
    #     """
    #     INSERT INTO courses 
    #     (course_code, course_name, program, level, semester)
    #     VALUES (?, ?, ?, ?, ?)
    #     """
    # ),
    "Student details": (
        "student_details",
        """
        INSERT INTO student_details 
        (student_id, name, surname, dob, gender, marital_status, address, next_of_kin, email, phone_number)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
    ),
    "records": (
        "records",
        """
        INSERT INTO records 
        (student_id, course_code, year_enrolled, semester, course_gpa, course_credits, year_of_study, academic_year, recarried, status, graduation, degree_class)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
    ),
    "academic records": (
        "academic_records",
        """
        INSERT INTO academic_records 
        (student_id, course_code, enrollment_year, year_of_study, semester, program, duration, gpa, credits_earned, status, graduation_year, course_gpa, course_credits, degree_class, recarried)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
    ),
    "Attendance and Engagement": (
        "attendance",
        """
        INSERT INTO attendance 
        (student_id, course_code, year_of_study, semester, physical_attendance_expected, attendance, attendance_percentage, inclass, presentations, assignments, lms_logins, engagement_score, participation_score)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
    )
}

# Function to get numeric column names from SQL Server schema
def get_numeric_columns(table_name):
    sql = f"""
    SELECT COLUMN_NAME, DATA_TYPE 
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = '{table_name}'
    """
    df_schema = pd.read_sql(sql, conn)
    numeric_cols = df_schema[df_schema["DATA_TYPE"].isin(
        ["int", "decimal", "numeric", "float", "real", "bit"]
    )]["COLUMN_NAME"].tolist()
    return numeric_cols

# Loop over sheets and insert into matching tables
for sheet_name, df in sheets.items():
    if sheet_name in sheet_to_table:
        table_name, sql = sheet_to_table[sheet_name]
        print(f"Importing {sheet_name} → {table_name}")

        # Replace NaN with None for SQL Server
        df = df.where(pd.notnull(df), None)

        # Get numeric columns for this table
        numeric_cols = get_numeric_columns(table_name)

        # Convert them safely to numbers
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors="coerce")

        # Special handling for BIT (convert Yes/No, True/False, etc.)
        for col in df.columns:
            if col in numeric_cols:
                if df[col].dtype == object:
                    df[col] = df[col].replace(
                        {"Yes": 1, "No": 0, "True": 1, "False": 0}
                    )
                # force to int where possible
                df[col] = df[col].astype("Int64")

        # Insert rows
        for row in df.itertuples(index=False):
            cursor.execute(sql, *row)

conn.commit()
conn.close()

print("Import finished for all sheets!")

